# 作业要求

使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少。

# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/mnt/ws/LLM-quickstart/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,1 star,"Disgusting! There is a bad smell right when you walk in. Don't order the burrito, worst I've ever had. The bathrooms smelled so bad I almost threw up. The only thing good was the server was nice and the wings were good."
1,2 star,"Not impressed. Ordered breakfast and instead of serving the toast right away, we had to wait until our meals were ready (which also took a long while). My table was not offered any coffee refills and basically the servers never returned to check on us. If they weren't bringing out the food, then they would be standing in the front. \nAs another yelper mentioned, the food was mediocre. There are many restaurants nearby with better service."
2,2 star,"Clearly Arizona does not know what good sushi is all about. I guess that shouldn't surprise me, considering it's nowhere near an ocean, but still I expected much better, especially for the prices.\n\nThe Upside: super hot waitstaff, surprisingly good calamari, cool modern feel.\n\nThe downside: overpriced, slow service, crowded and uncomfortable patio area, barely edible sushi rolls (seriously, I've had better from the grocery store deli).\n\nCome here for some sake and appetizers, but eat dinner someplace else. And don't go to the place across the street! It's the same owners."
3,4 stars,"As for sushi buffets..this one gets my top rating.\n\nNot your typical 'california rolls' buffet with a few odd things added in. They have real sushi chefs working behind a large bar. The sushi is extreamly FRESH, replaced often. The Tuna is just to die for, nothing beats the flavor of fresh fish! Doesn't smell fishy either. They have their share of cooked items (Calamari, soup, beef and the like). My one suggestion is to please lable each item, some, or most things are hard to tell what's in 'em. Not a place for the weak of sushi eaters...they have some Real down and dirty RAW Fish!! Yummy all around!\nThe price is as good as any other All You can eat, but you don't have to wait to order. Walk right up and serve yourself! Dinner is around $25 per person!\n\nThis place is clean and large enough for the whole family!"
4,1 star,"Cold, hard and old\nWent to Hot Bagels twice, first time near closing time. The bagels were cold, hard and stale, had to throw it away.\nGiving them a second chance, I went at 10 am since they say they make them everyday. Ordered 6 and my home is no more than 5 min away. Once again, cold, hard and stale.\nCalled and they said they make them once a day early in the morning. If they leave them out, that explains a lot. If they put them in the refrigerator, then I guess they are not aware that bread stales 6 times faster in the refrigerator.\nEither way, I will never go back.\n\nI can't comment about the other stuff they sell, but as for bagels, KEEP AWAY, unless you have strong teeth."
5,3 stars,"A favorite lunch spot of mine. If you get there before 11:30 am they have a \""recession friendly\"" menu where almost everything on the lunch menu is $5.95. Quite a deal considering the portions."
6,1 star,"I hate writing negative reviews.. i really do, but man oh man, Gus's Diner is not something that lends itself to positive reviews.\n\nFood- gross, unhealthy\nCleanliness- dirty\nService- 2 thumbs down\nAmbiance- almost everyone in this place is pushing 350lbs... and shoveling food down their throats, as to keep that standard going. it's not a comfortable place to eat\n\nI have nothing nice to say.."
7,5 stars,"One of the best burgers I've had in Montreal so far. Deville's take on burgers are definitely not run of the mill. My boyfriend had the inside out, it looked like a reverse flower bloom on a bun, and he loved the taste. A very friendly waitress recommended The Fat Cat burger and she wasn't wrong about the juiciness. I definitely recommend giving this place a try if you're in the mood for a fun decor and burgers."
8,3 stars,"Every time I walk through the big glass doors of PF Changs, I hate myself a little more. I tell myself,

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/mnt/ws/LLM-quickstart/.conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,4 stars,"Used to work at Peter Lik WAY up there on the 3rd floor. \""There's a 3rd floor in the Forum Shops?\"" And thus place makes an amazing pumpkin sliced mocha (in season) and some bomb Mochas any time )","[101, 17627, 1106, 1250, 1120, 1943, 5255, 1377, 22751, 3663, 1146, 1175, 1113, 1103, 2973, 1837, 119, 165, 107, 1247, 112, 188, 170, 2973, 1837, 1107, 1103, 8576, 13125, 1116, 136, 165, 107, 1262, 2456, 1282, 2228, 1126, 6929, 11188, 4314, 17053, 182, 9962, 1161, 113, 1107, 1265, 114, 1105, 1199, 5985, 12556, 7147, 1116, 1251, 1159, 114, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(325000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(25000))
#small_train_dataset = tokenized_datasets["train"]
#small_eval_dataset = tokenized_datasets["test"]
len(small_train_dataset),len(small_eval_dataset)

(325000, 25000)

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [11]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
# 650000 * 3 / 128 = 15234 
# 15234 / 1000 = 16
# training_args = TrainingArguments(output_dir=model_dir,
#                                   per_device_train_batch_size=128,
#                                   num_train_epochs=3,
#                                   logging_steps=1000)

In [12]:
# 完整的超参数配置
# print(training_args)

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [13]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [15]:
from transformers import TrainingArguments, Trainer

model_dir = "models/bert-base-cased-finetune-yelp"
# 650000 * 3 / 20 = 97500
# 97500 / 1000 = 98
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=20,
                                  num_train_epochs=3,
                                  logging_steps=1000)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [17]:
trainer.train()

  2%|▏         | 1000/48750 [15:33<12:17:32,  1.08it/s]

{'loss': 1.012, 'learning_rate': 4.8974358974358975e-05, 'epoch': 0.06}


  4%|▍         | 2000/48750 [31:05<12:07:38,  1.07it/s]

{'loss': 0.9043, 'learning_rate': 4.7948717948717955e-05, 'epoch': 0.12}


  6%|▌         | 3000/48750 [45:42<10:58:23,  1.16it/s]

...

 98%|█████████▊| 48000/48750 [11:46:52<10:46,  1.16it/s]

{'loss': 0.5375, 'learning_rate': 7.692307692307694e-07, 'epoch': 2.95}


                                                        
100%|██████████| 48750/48750 [12:03:54<00:00,  1.12it/s]

{'eval_loss': 0.7833148241043091, 'eval_accuracy': 0.68344, 'eval_runtime': 371.5408, 'eval_samples_per_second': 67.287, 'eval_steps_per_second': 8.411, 'epoch': 3.0}
{'train_runtime': 43434.0735, 'train_samples_per_second': 22.448, 'train_steps_per_second': 1.122, 'train_loss': 0.6927453901241987, 'epoch': 3.0}


TrainOutput(global_step=48750, training_loss=0.6927453901241987, metrics={'train_runtime': 43434.0735, 'train_samples_per_second': 22.448, 'train_steps_per_second': 1.122, 'train_loss': 0.6927453901241987, 'epoch': 3.0})

In [24]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(1000))

In [25]:
trainer.evaluate(small_test_dataset)

100%|██████████| 125/125 [00:15<00:00,  8.05it/s]


{'eval_loss': 0.8090919256210327,
 'eval_accuracy': 0.677,
 'eval_runtime': 15.6756,
 'eval_samples_per_second': 63.793,
 'eval_steps_per_second': 7.974,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [20]:
trainer.save_model(model_dir)

In [21]:
trainer.save_state()

In [23]:
# trainer.model.save_pretrained("./")

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少